In [14]:
import pandas as pd
import numpy as np
import os
import json

# 读取数据
root_path = "/mnt/pfs-mc0p4k/cv/team/panxuhao/playground/repo/med_proj"
file_path = f"{root_path}/data/all_data.xlsx"
data = pd.read_excel(file_path)

id2cluster = pd.read_csv(f"{root_path}/data/48h/result.csv", index_col=None, header=0)
with open(f"{root_path}/results/dataprocess/48h/name_id_mapping.json", "r", encoding="utf-8") as f:
    name2id = json.load(f)
id2name = {v: k for k, v in name2id.items()}

name2cluster = {}
for index, row in id2cluster.iterrows():
    name2cluster[id2name[row['ID']]] = row['cluster']

In [15]:
with open(f"{root_path}/results/anova/48h/name2cluster.txt", "w", encoding="utf-8") as f:
    for name, cluster in name2cluster.items():
        f.write(f"{name}, {cluster}\n")

In [16]:
root_path = "/mnt/pfs-mc0p4k/cv/team/panxuhao/playground/repo/med_proj"
file_path = f"{root_path}/data/all_data.xlsx"
data = pd.read_excel(file_path)
data["转归"].value_counts()

死亡    200
存活    126
Name: 转归, dtype: int64

In [17]:
df_filtered = data[data['姓名'].isin(name2id.keys())]

# 将 name2id 中的对应ID添加到新列中
df_filtered['cluster'] = df_filtered['姓名'].map(name2cluster)
df_filtered.rename(columns={'姓名': 'name'}, inplace=True)
df_filtered['性别'] = df_filtered['性别'].replace({'男': 1, '女': 0})
df_filtered['转归'] = df_filtered['转归'].replace({'存活': 0,'死亡': 1, '好转':0, "未愈": 1, "其他":0})
df_filtered['机械通气'] = df_filtered['机械通气'].replace({'无': 0, '有': 1})
df_filtered['是否二感'] = df_filtered['是否二感'].replace({'否': 0, '是': 1, '疑似': 0})
df_filtered['感染部位拼接'] = df_filtered['入院感染部位'] + df_filtered['新感部位']
df_filtered['慢性病个数'] = df_filtered['慢性病个数'].replace({0: 0, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1})

df_filtered['感染部位拼接'] = df_filtered['感染部位拼接'].apply(lambda x: 1 if '血液' in x else 0)
df_filtered['入院感染部位'] = df_filtered['入院感染部位'].apply(lambda x: 1 if '血液' in x else 0)
df_filtered['新感部位'] = df_filtered['新感部位'].apply(lambda x: 1 if '血液' in x else 0)

/tmp/ipykernel_651127/1950129209.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['cluster'] = df_filtered['姓名'].map(name2cluster)
/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/tmp/ipykernel_651127/1950129209.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [18]:
df_filtered.to_csv(f'{root_path}/results/anova/48h/cluster_feature_adjust.csv', index=False)

### 使用方差分析得到结结果

In [19]:
from scipy.stats import f_oneway
import numpy as np

data = df_filtered
# Extracting columns for ANOVA, excluding 'name' and 'cluster' columns
anova_columns = data.columns.drop(['name', 'cluster'])

# Dictionary to hold ANOVA results
anova_results = {}

# Performing ANOVA for each column and storing the results
for column in anova_columns:
    groups = data.groupby('cluster')[column].apply(list)
    f_val, p_val = f_oneway(*groups)
    anova_results[column] = {'F-value': f_val, 'p-value': p_val}

# Convert ANOVA results to a DataFrame
anova_df = pd.DataFrame(anova_results).T

# Calculating mean values for each column per cluster
mean_values = data.groupby('cluster')[anova_columns].mean()

# Calculating median values for each column per cluster
median_values = data.groupby('cluster')[anova_columns].median()

# Calculating count of each column per cluster
group_counts = data.groupby('cluster').size().reset_index(name='count')

anova_df.to_csv(f'{root_path}/results/anova/48h/anova.csv', encoding="utf-8-sig")
mean_values.to_csv(f'{root_path}/results/anova/48h/mean_values.csv', encoding="utf-8-sig")
median_values.to_csv(f'{root_path}/results/anova/48h/median_values.csv', encoding="utf-8-sig")
group_counts.to_csv(f'{root_path}/results/anova/48h/group_counts.csv', encoding="utf-8-sig")

#### 比较其中一组和其他三组的差异

In [7]:
from scipy.stats import f_oneway
import pandas as pd

# 假设 df_filtered 是你的数据框
data = df_filtered

# 初始化结果存储字典
anova_results_by_cluster = {}

# 需要比较的 cluster 列表
clusters_to_compare = [1, 2, 3]

# 遍历除了'name'和'cluster'外的所有列
for column in data.columns.drop(['name', 'cluster']):
    # 存储当前列的比较结果
    column_results = {}
    
    # 对每个需要比较的 cluster 执行 ANOVA
    for compare_cluster in clusters_to_compare:
        # 提取 cluster = 4 和当前比较 cluster 的数据
        group1 = data[data['cluster'] == 4][column]
        group2 = data[data['cluster'] == compare_cluster][column]

        # 执行 ANOVA
        f_val, p_val = f_oneway(group1, group2)
        column_results[f'cluster_4_vs_{compare_cluster}'] = {'F-value': f_val, 'p-value': p_val}
    
    # 将当前列的结果存入总字典
    anova_results_by_cluster[column] = column_results

# 将结果转换为更方便阅读的形式
anova_df = pd.DataFrame.from_dict({(i, j): anova_results_by_cluster[i][j] 
                                   for i in anova_results_by_cluster.keys() 
                                   for j in anova_results_by_cluster[i].keys()},
                                  orient='index')

# 可以将结果保存为 CSV 文件
anova_df.to_csv(f'{root_path}/results/anova/48h/cluster_4_comparisons.csv', encoding="utf-8-sig")


#### 两两之间的差异

In [23]:
from scipy.stats import f_oneway
import pandas as pd
import itertools

# 假设 df_filtered 是你的数据框
data = df_filtered

# 初始化结果存储字典
anova_results_by_cluster = {}

# 获取所有 unique 的 cluster 值
clusters = data['cluster'].unique()

# 遍历除了 'name' 和 'cluster' 外的所有列
for column in data.columns.drop(['name', 'cluster']):
    # 存储当前列的比较结果
    column_results = {}
    
    # 对每个可能的 cluster 组合执行 ANOVA
    for (cluster1, cluster2) in itertools.combinations(clusters, 2):
        # 提取 cluster1 和 cluster2 的数据
        group1 = data[data['cluster'] == cluster1][column]
        group2 = data[data['cluster'] == cluster2][column]

        # 执行 ANOVA
        f_val, p_val = f_oneway(group1, group2)
        column_results[f'cluster_{cluster1}_vs_{cluster2}'] = {'F-value': f_val, 'p-value': p_val}
    
    # 将当前列的结果存入总字典
    anova_results_by_cluster[column] = column_results

# 将结果转换为更方便阅读的形式
anova_df = pd.DataFrame.from_dict({(i, j): anova_results_by_cluster[i][j] 
                                   for i in anova_results_by_cluster.keys() 
                                   for j in anova_results_by_cluster[i].keys()},
                                  orient='index')

# 可以将结果保存为 CSV 文件
anova_df.to_csv(f'{root_path}/results/anova/48h/all_clusters_comparisons.csv', encoding="utf-8-sig")
